In [ ]:
# 20231026回家作業說明
# 到環境部抓取aqx_p_07空氣品質監測站基本資料
# 參考網址 https://data.moenv.gov.tw/swagger/#/%E5%A4%A7%E6%B0%A3/get_aqx_p_07
# 建立password.py,提供api key,
# 建立連結抓取檔案（要確認csv檔或是json檔),
# 建立資料庫存取抓下來的資料
# 建立執行緒，進行每小時抓取一次資料

In [61]:
import password #匯入並執行password.py檔
import requests #匯入並執行requests
import sqlite3 #匯入sqlite3模組
import json #匯入json模組
#def get_air_data():
url=f'https://data.moenv.gov.tw/api/v2/aqx_p_07?api_key={password.apikey}'
print(url) #驗證url是否正確
res=requests.get(url)
data=res.json()['records']
print(data)




https://data.moenv.gov.tw/api/v2/aqx_p_07?api_key=f5bebf68-0b35-4a9f-b7ee-58e4fd9cf480
[{'sitename': '大城', 'siteengname': 'Dacheng', 'areaname': '中部空品區', 'county': '彰化縣', 'township': '大城鄉', 'siteaddress': '彰化縣大城鄉西厝路98號', 'twd97lon': '120.26964167', 'twd97lat': '23.85493056', 'sitetype': '其它站', 'siteid': '85'}, {'sitename': '富貴角', 'siteengname': 'FugueiCape', 'areaname': '北部空品區', 'county': '新北市', 'township': '石門區', 'siteaddress': '', 'twd97lon': '121.53656894', 'twd97lat': '25.29681695', 'sitetype': '背景站', 'siteid': '84'}, {'sitename': '麥寮', 'siteengname': 'Mailiao', 'areaname': '雲嘉南空品區', 'county': '雲林縣', 'township': '麥寮鄉', 'siteaddress': '雲林縣麥寮鄉中興路115號', 'twd97lon': '120.251825', 'twd97lat': '23.753506', 'sitetype': '工業站', 'siteid': '83'}, {'sitename': '關山', 'siteengname': 'Guanshan', 'areaname': '花東空品區', 'county': '臺東縣', 'township': '關山鎮', 'siteaddress': '臺東縣關山鎮自強路66號', 'twd97lon': '121.161933', 'twd97lat': '23.045083', 'sitetype': '其它站', 'siteid': '80'}, {'sitename': '馬公', 'siteengna

In [ ]:
#def create_table_air(conn:sqlite3.Connection):
conn=sqlite3.Connection(database='air.db')
cursor=conn.cursor()
cursor.execute(
    '''
    CREATE TABLE IF NOT EXISTS AIR(
    "ID"    INTEGER,
    "監測站名"  TEXT NOT NULL,
    "空品分區"  TEXT NOT NULL,
    "縣市行政區"    TEXT NOT NULL,
    "監測站地址"    TEXT NOT NULL,
    "監測站類型"    TEXT NOT NULL,
    "監測站編號"    TEXT NOT NULL,
    PRIMARY KEY("ID" AUTOINCREMENT)
    );
    '''
)
conn.commit()